In [1]:
import torch, torch.nn as nn
import snntorch as snn
from snntorch import surrogate
import snntorch.functional as SF

In [2]:
class SNNModel(torch.nn.Module):
    def __init__(self):
        super(SNNModel, self).__init__()

        beta = 0.9  # neuron decay rate 
        spike_grad = surrogate.fast_sigmoid() # fast sigmoid surrogate gradient


        self.linearIn = nn.Linear(6, 10)
        self.spike1 = snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True)
        self.linearOut = nn.Linear(10, 27)
        self.spike2 = snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True)

        self.men = torch.zeros(1)
        self.spk = torch.zeros(1)
        self.steps = 4096;

    def forward(self, x):
        for step in range(self.steps):
            self.spk = self.linearIn(x[step])
            self.spk, self.men = self.spike1(self.spk, self.mem)
            self.spk = self.linearOut()
            self.spk, self.men = self.spike2(self.spk, self.mem)
        return self.spk


In [3]:
model = SNNModel()
# Ensures that the networks runs with floats
model.float()

SNNModel(
  (linearIn): Linear(in_features=6, out_features=10, bias=True)
  (spike1): Leaky()
  (linearOut): Linear(in_features=10, out_features=27, bias=True)
  (spike2): Leaky()
)

In [4]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Define the parameters
batch_size = 4
num_channels = 6
sequence_length = 4096

# Generate fake training data
training_data = torch.randn((batch_size, num_channels, sequence_length))

# Print the shape of the generated data
print("Shape of training data:", training_data.shape)

Shape of training data: torch.Size([4, 6, 4096])


In [5]:
model = SNNModel()
utils.reset(model)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3, betas=(0.9, 0.999))
loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)
num_epochs = 1

for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(training_data)
    loss = loss_fn(output, target_data)
    loss.backward()
    optimizer.step()

NameError: name 'utils' is not defined